In [10]:
import pandas as pd
import numpy as np

In [2]:
path_corpus = "/Users/philippmetzger/OneDrive/PHILIPP/NOVA IMS/2nd Semester/06 Text Mining 4 ECTS/00 Project/corpus/"
cs_en = pd.read_csv(path_corpus + "cs-en/scores.csv")
de_en = pd.read_csv(path_corpus + "de-en/scores.csv")
en_fi = pd.read_csv(path_corpus + "en-fi/scores.csv")
en_zh = pd.read_csv(path_corpus + "en-zh/scores.csv")
ru_en = pd.read_csv(path_corpus + "ru-en/scores.csv")
zh_en = pd.read_csv(path_corpus + "zh-en/scores.csv")

In [11]:
descriptions = ["Russian into English", "German into English", "Czech into English", "Chinese into English", "English into Chinese", "English into Finish"]

rows = []
zscores = []
avgscores = []
annots = []

i = 0

for element in [ru_en, de_en, cs_en, zh_en, en_zh, en_fi]:
    rows.append(element.shape[0])
    zscores.append(np.round(element["z-score"].mean(),2))
    avgscores.append(np.round(element["avg-score"].mean(), 2))
    annots.append(np.round(element["annotators"].mean(),2))
    i += 1                   
    
exploration_df = pd.DataFrame([rows, zscores, avgscores, annots]).T.rename(columns={0:"rows", 1:"avg z-score", 2:"avg avg-score", 3:"avg annotators"})
exploration_df["description"] = descriptions
exploration_df = exploration_df.set_index("description")
exploration_df

,rows,avg z-score,avg avg-score,avg annotators
description,,,,
Russian into English,17980.0,0.01,74.50,1.30
German into English,21704.0,0.00,71.85,1.50
Czech into English,11585.0,-0.03,69.24,1.89
Chinese into English,26419.0,-0.05,66.06,1.42
English into Chinese,10221.0,-0.06,65.98,1.58
English into Finish,6748.0,-0.14,45.12,1.23


In [13]:
de_en.head(3)

,source,reference,translation,z-score,avg-score,annotators
0,"Ihr Zeitlupentempo maßen sie, als sie vor Spit...",Her timeless pace measures them when they equi...,Their slow speed was measured by researchers o...,-0.345024,76.0,1
1,"Er sagte, dass die Bereiche ruhige Treffpunkte...",He said the areas offer quiet meeting points b...,He said the spaces provided calm meeting point...,0.903800,97.5,2
2,Für die Geschäftsleute an der B 27 ist es nur ...,"For businessmen at the B 27, it's only a small...",This is only a small consolation for businesse...,0.700503,94.0,1


# Word Mover's distance
## https://towardsdatascience.com/word-movers-distance-for-text-similarity-7492aeca71b0

In [192]:
from nltk.corpus import stopwords
from nltk import download
import re
import torch
import torchtext.vocab as vocab
from tqdm import tqdm_notebook as tqdm
from scipy.stats import pearsonr
from scipy.stats import kendalltau

In [92]:
glove = vocab.GloVe(name='6B', dim=50)
print('Loaded {} words'.format(len(glove.itos)))

.vector_cache/glove.6B.zip: 862MB [03:26, 4.18MB/s]                               
100%|█████████▉| 399999/400000 [00:09<00:00, 44101.47it/s]


Loaded 400000 words


In [93]:
def get_word(word):
    return glove.vectors[glove.stoi[word]]

In [99]:
download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philippmetzger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [202]:
stop = stopwords.words('english')

In [196]:
distances = []

for i in tqdm(range(de_en.shape[0])):
    
    over = False

    a = de_en.iloc[i,1]
    b = de_en.iloc[i,2]

    a = a.lower()
    a = re.sub("[^a-z]", ' ', a)
    a = a.split()

    b = b.lower()
    b = re.sub("[^a-z]", ' ', b)
    b = b.split()

    a = [word for word in a if word not in stop]
    b = [word for word in b if word not in stop]

    len_pre_oov1 = len(a)
    len_pre_oov2 = len(b)
    a = [token for token in a if token in glove.itos]
    b = [token for token in b if token in glove.itos]
    diff1 = len_pre_oov1 - len(a)
    diff2 = len_pre_oov2 - len(b)

    if len(a) == 0 or len(b) == 0:
        #print('At least one of the documents had no words that were in the vocabulary. Aborting (returning inf).')
        dist = -1
        over = True

    if not over:
        dist = 0
        for word_a in a:
            distances_this = []
            for word_b in b:

                distances_this.append(torch.dist(get_word(word_a), get_word(word_b)))

            min_dist = min(distances_this)

            dist += min_dist
    
    distances.append(dist)

max_dist = max(distances)
distances = [max_dist if dist == -1 else dist for dist in distances]

<ipython-input-196-c82a6a2e59d3>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(de_en.shape[0])):


  0%|          | 0/21704 [00:00<?, ?it/s]

In [197]:
pearsonr(de_en['avg-score'], distances)

(-0.1531166657477652, 5.600086123175844e-114)

In [198]:
kendalltau(de_en['avg-score'], distances)

KendalltauResult(correlation=-0.13532525320110378, pvalue=2.355091701399975e-193)

In [199]:
distances = []

for i in tqdm(range(de_en.shape[0])):
    
    over = False

    a = de_en.iloc[i,1]
    b = de_en.iloc[i,2]

    a = a.lower()
    a = re.sub("[^a-z]", ' ', a)
    a = a.split()

    b = b.lower()
    b = re.sub("[^a-z]", ' ', b)
    b = b.split()

    #a = [word for word in a if word not in stop]
    #b = [word for word in b if word not in stop]

    len_pre_oov1 = len(a)
    len_pre_oov2 = len(b)
    a = [token for token in a if token in glove.itos]
    b = [token for token in b if token in glove.itos]
    diff1 = len_pre_oov1 - len(a)
    diff2 = len_pre_oov2 - len(b)

    if len(a) == 0 or len(b) == 0:
        #print('At least one of the documents had no words that were in the vocabulary. Aborting (returning inf).')
        dist = -1
        over = True

    if not over:
        dist = 0
        for word_a in a:
            distances_this = []
            for word_b in b:

                distances_this.append(torch.dist(get_word(word_a), get_word(word_b)))

            min_dist = min(distances_this)

            dist += min_dist
    
    distances.append(dist)

max_dist = max(distances)
distances = [max_dist if dist == -1 else dist for dist in distances]

<ipython-input-199-b14bd5e79b56>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(de_en.shape[0])):


  0%|          | 0/21704 [00:00<?, ?it/s]

In [200]:
pearsonr(de_en['avg-score'], distances)

(-0.16483615596895962, 5.015035993248079e-132)

In [201]:
kendalltau(de_en['avg-score'], distances)

KendalltauResult(correlation=-0.1396207216132029, pvalue=4.852868577639179e-206)